# [Q-DRANT](https://qdrant.tech/documentation/quickstart/)

## Setup

### Run QDrant Using Docker

```sh
docker run -p 6333:6333 -p 6334:6334 \
    -v $(pwd)/qdrant_storage:/qdrant/storage:z \
    --name qdrant-db --rm \
    qdrant/qdrant
```

In [1]:
# Built-in library
from pathlib import Path
import re
import json
from typing import Any, Literal, Optional, Union
import logging
import warnings

# Standard imports
import numpy as np
import numpy.typing as npt
from pprint import pprint
import pandas as pd
import polars as pl
from rich.console import Console
from rich.theme import Theme

custom_theme = Theme(
    {
        "white": "#FFFFFF",  # Bright white
        "info": "#00FF00",  # Bright green
        "warning": "#FFD700",  # Bright gold
        "error": "#FF1493",  # Deep pink
        "success": "#00FFFF",  # Cyan
        "highlight": "#FF4500",  # Orange-red
    }
)
console = Console(theme=custom_theme)

# Visualization
# import matplotlib.pyplot as plt

# NumPy settings
np.set_printoptions(precision=4)

# Pandas settings
pd.options.display.max_rows = 1_000
pd.options.display.max_columns = 1_000
pd.options.display.max_colwidth = 600

# Polars settings
pl.Config.set_fmt_str_lengths(1_000)
pl.Config.set_tbl_cols(n=1_000)
pl.Config.set_tbl_rows(n=1_000)

warnings.filterwarnings("ignore")

# Black code formatter (Optional)
%load_ext lab_black

# auto reload imports
%load_ext autoreload
%autoreload 2

In [2]:
def go_up_from_current_directory(*, go_up: int = 1) -> None:
    """This is used to up a number of directories.

    Params:
    -------
    go_up: int, default=1
        This indicates the number of times to go back up from the current directory.

    Returns:
    --------
    None
    """
    import os
    import sys

    CONST: str = "../"
    NUM: str = CONST * go_up

    # Goto the previous directory
    prev_directory = os.path.join(os.path.dirname(__name__), NUM)
    # Get the 'absolute path' of the previous directory
    abs_path_prev_directory = os.path.abspath(prev_directory)

    # Add the path to the System paths
    sys.path.insert(0, abs_path_prev_directory)
    print(abs_path_prev_directory)

In [3]:
go_up_from_current_directory(go_up=2)

from QA_and_RAG import PACKAGE_ROOT_PATH
from QA_and_RAG.src.utils.utilities import ProcessFiles
from config import config, settings

/Users/neidu/Desktop/Projects/Personal/My_Projects/Gen-AI-Projects


In [4]:
from qdrant_client import QdrantClient, models
from sentence_transformers import SentenceTransformer

In [5]:
model_name_or_path: str = "all-MiniLM-L6-v2"
encoder: SentenceTransformer = SentenceTransformer(model_name_or_path)

encoder.get_sentence_embedding_dimension()

384

### Add Dataset

In [6]:
fp: str = "../data/vector_db_data/music-sample-data.csv"
df: pl.DataFrame = pl.read_csv(fp)
# .with_columns(
#     metadata=pl.concat_str(
#         ["name", "sex", "ticket"],
#         separator=" || ",
#     )
# )

sample_df: pl.DataFrame = df.sample(df.shape[0], seed=1)
sample_df.head()

title,description,genre,artist,year
str,str,str,str,i64
"""'Imagine'""","""'A peaceful anthem advocating for world peace'""","""'Rock'""","""'John Lennon'""",1971
"""'Billie Jean'""","""'A pop classic with a memorable bassline'""","""'Pop'""","""'Michael Jackson'""",1982
"""'Juicy'""","""'A classic East Coast rap anthem'""","""'Rap'""","""'The Notorious B.I.G.'""",1994
"""'Still D.R.E.'""","""'A comeback rap song with a iconic beat'""","""'Rap'""","""'Dr. Dre'""",1999
"""'Sweet Child O' Mine'""","""'A hard rock anthem with a distinctive opening riff'""","""'Rock'""","""'Guns N' Roses'""",1987


In [7]:
documents: list[dict[str, Any]] = sample_df.to_dicts()
sample_df.to_dicts()[:2]

[{'title': "'Imagine'",
  'description': "'A peaceful anthem advocating for world peace'",
  'genre': "'Rock'",
  'artist': "'John Lennon'",
  'year': 1971},
 {'title': "'Billie Jean'",
  'description': "'A pop classic with a memorable bassline'",
  'genre': "'Pop'",
  'artist': "'Michael Jackson'",
  'year': 1982}]

### Create Client And Collection

In [8]:
from qdrant_client.http.exceptions import ResponseHandlingException

In [9]:
# host: str = "0.0.0.0"
# port: int = 6333
# api_key: str | None = None
# client = QdrantClient(url="http://localhost:6333")

client: QdrantClient = QdrantClient(
    host=settings.QDRANT_HOST,
    port=settings.QDRANT_PORT,
    api_key=settings.QDRANT_API_KEY.get_secret_value(),
    https=False,
)


try:
    client.get_collections().collections
    print("Qdrant server is running.")
except ResponseHandlingException as e:
    print(f"Qdrant server is not running. Error: {e}")

Qdrant server is running.


In [10]:
# Create collection
collection_name: str = "music_collection"
embedding_size: int = encoder.get_sentence_embedding_dimension()

if collection_name in [col.name for col in client.get_collections().collections]:
    print(f"Collection '{collection_name}' already exists.")

else:
    print(f"Creating collection '{collection_name}'.")
    client.create_collection(
        collection_name=collection_name,
        vectors_config=models.VectorParams(
            size=embedding_size, distance=models.Distance.COSINE
        ),
    )

Collection 'music_collection' already exists.


In [11]:
if client.collection_exists(collection_name=collection_name):
    print("Database exists")
else:
    print("Database does not exist")

Database exists


### Upload Data to Qdrant

In [12]:
res = encoder.encode("This is a test.")

res.tolist()[:2]

[0.03359076753258705, 0.010512457229197025]

In [13]:
def embed_document(document: str) -> list[float]:
    """Embed a document using an embedding model."""
    return encoder.encode(document).tolist()


len(embed_document("This is a test."))

384

In [ ]:
client.upsert(
    collection_name=collection_name,
    points=[
        models.PointStruct(
            id=idx, vector=embed_document(doc["description"]), payload=doc
        )
        for idx, doc in enumerate(documents)
    ],
)

### Query The Vector Database

In [14]:
query: str = "rap battle"

hits = client.query_points(
    collection_name=collection_name, query=embed_document(query), limit=3
)
hits.points

[ScoredPoint(id=3, version=0, score=0.5759016, payload={'title': "'Still D.R.E.'", 'description': "'A comeback rap song with a iconic beat'", 'genre': "'Rap'", 'artist': "'Dr. Dre'", 'year': 1999}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=5, version=0, score=0.56217456, payload={'title': "'Lose Yourself'", 'description': "'An intense rap song with a powerful message'", 'genre': "'Rap'", 'artist': "'Eminem'", 'year': 2002}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=27, version=0, score=0.54559654, payload={'title': "'Old Town Road'", 'description': "'A country-rap fusion with a viral appeal'", 'genre': "'Country'", 'artist': "'Lil Nas X'", 'year': 2019}, vector=None, shard_key=None, order_value=None)]

In [15]:
print(f"Query: {query}")
for hit in hits.points:
    console.print(hit.payload, "score: ", hit.score)

Query: rap battle


{
    'title': "'Still D.R.E.'",
    'description': "'A comeback rap song with a iconic beat'",
    'genre': "'Rap'",
    'artist': "'Dr. Dre'",
    'year': 1999
}
score:  0.5759016

{
    'title': "'Lose Yourself'",
    'description': "'An intense rap song with a powerful message'",
    'genre': "'Rap'",
    'artist': "'Eminem'",
    'year': 2002
}
score:  0.56217456

{
    'title': "'Old Town Road'",
    'description': "'A country-rap fusion with a viral appeal'",
    'genre': "'Country'",
    'artist': "'Lil Nas X'",
    'year': 2019
}
score:  0.54559654

#### Add Filters

- Narrow down the search results by adding filters to the query.

In [16]:
query_filter = models.Filter(
    must=[models.FieldCondition(key="year", range=models.Range(gte=2_000))]
)
# OR
query_filter = {
    "must": [
        {"key": "year", "range": {"gte": 2_000}},
    ]
}

hits = client.query_points(
    collection_name=collection_name,
    query=embed_document(query),
    query_filter=query_filter,
    limit=3,
)
hits

QueryResponse(points=[ScoredPoint(id=5, version=0, score=0.56217456, payload={'title': "'Lose Yourself'", 'description': "'An intense rap song with a powerful message'", 'genre': "'Rap'", 'artist': "'Eminem'", 'year': 2002}, vector=None, shard_key=None, order_value=None), ScoredPoint(id=27, version=0, score=0.54559654, payload={'title': "'Old Town Road'", 'description': "'A country-rap fusion with a viral appeal'", 'genre': "'Country'", 'artist': "'Lil Nas X'", 'year': 2019}, vector=None, shard_key=None, order_value=None), ScoredPoint(id=14, version=0, score=0.45459324, payload={'title': "'This Is America'", 'description': "'A socially conscious rap song with a powerful video'", 'genre': "'Rap'", 'artist': "'Childish Gambino'", 'year': 2018}, vector=None, shard_key=None, order_value=None)])

In [17]:
print(f"Query: {query}")
for hit in hits.points:
    console.print(hit.payload, "score: ", hit.score)

Query: rap battle


{
    'title': "'Lose Yourself'",
    'description': "'An intense rap song with a powerful message'",
    'genre': "'Rap'",
    'artist': "'Eminem'",
    'year': 2002
}
score:  0.56217456

{
    'title': "'Old Town Road'",
    'description': "'A country-rap fusion with a viral appeal'",
    'genre': "'Country'",
    'artist': "'Lil Nas X'",
    'year': 2019
}
score:  0.54559654

{
    'title': "'This Is America'",
    'description': "'A socially conscious rap song with a powerful video'",
    'genre': "'Rap'",
    'artist': "'Childish Gambino'",
    'year': 2018
}
score:  0.45459324

## Putting It Together

In [41]:
from QA_and_RAG.src.db_utils import VectorDBManager


files_dir: str = f"{config.QA_and_RAG.path}/data/vector_db_data/"
vector_db_manager: VectorDBManager = VectorDBManager(files_dir=files_dir)
vector_db_manager.run()

Processing file: /Users/neidu/Desktop/Projects/Personal/My_Projects/Gen-AI-Projects/QA_and_RAG/data/vector_db_data/music-sample-data.csv
Qdrant server is running.
Collection 'music-sample-data' already exists.
Data from 'music-sample-data' is saved into the vector database.
All csv/parquet files are saved into the vector database.


('',
 [{'role': 'assistant',
   'content': 'Uploaded files are ready. Please ask your question'}])

In [42]:
# vector_db_manager.encoder

vector_db_manager._get_collection_names()

['titanic', 'music-sample-data', 'music_collection']

In [ ]:
# To Do
# 1. Add chunking to the vector store
# 2. Add filters to the vector store
# 3. Create the RAG pipeline.

Creating collection 'neidu_collection'.
